<h1> 이진 분류 예제 - 영화 분류

IMDB 데이터셋

인터넷 영화 데이터베이스에서 가져온 양극단의 리뷰 50,000개로 이루어진 데이터셋
훈련 데이터 25,000개 테스트 데이터 25,000
50% 긍정 50% 부정
각 리뷰 -단어 시퀀스가 숫자 시퀀스로 변환되어 있다. 각 숫자는 사전에 있는 고유한 단어를 나타낸다. 

In [2]:
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels)=imdb.load_data(num_words=10000)

num_words=1000은 훈련 데이터에서 가장 자주 나타나는 단어 10,000개만 사용하겠다는 의미
train_data와 test_data는 리뷰의 목록 
각 리뷰는 단어 인덱스의 리스트
train_labels와 test_labels는 부정을 나타내는 0과 긍정을 나타내는 1의 리스트. 

## 데이터 준비


* 같은 길이가 되도록 리스트에 패딩 추가 
(samples, sequence_length) 크기의 정수 텐서로 변환

* 패딩이란, 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(예를 들면 0)으로 채우는 기법. 출력 크기를 조정할 목적으로 사용. 
(4,4)*(3,3)필터 = (2,2) 이면 합성곱 연산을 되풀이할 때 크기가 작아짐. 그래서 (4,4)-> (6,6)으로 만들어 결과가 (4,4)가 되도록 조정

* 이 정수 텐서를 다룰 수 있는 층을 신경망의 첫 번째 층으로 사용. 이것을 `Embedding` 이라고 부름. 

* 리스트를 원-핫 인코딩하여 0과 1의 벡터로 변환한다. One-Hot Encoding이란 딥러닝을 이용한 자연어 처리에 사용되는 기법
먼저 단어집합(vocabulary)을 만들고 각각에 넘버링을 한다. 기법에 대한 설명은 나중에 공부하도록 하자. 

* 10,000차원의 벡터로 각각 변환하는데 그때 인덱스 3과 5의 위치만 1이고 나머지는 0인 것으로 한다. 

* 밑에서는 직접 데이터를 one-hot vector로 만들었다.     


In [5]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences): # 특정 인덱스만 1
        results[i, sequence] = 1.
    return results

#훈련 데이터를 벡터로 변환
x_train = vectorize_sequences(train_data)
#테스트 데이터를 벡터로 변환
x_test = vectorize_sequences(test_data)

In [7]:
# 레이블을 벡터로 바꿉니다
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

## 신경망 만들기 

입력 데이터가 벡터이고 레이블은 스칼라(1/0)이다. 이런 문제에 작 작동하는 네트워크 종류는 `relu` 활성화 함수를 사용한 완전 연결층(즉, `Dense(16,activation='relu))`을 그냥 쌓은 것
* relu함수가 등장하기 전에 있었던 시그모이드 활성화 함수 같은 경우 레이어를 거칠 수록 값이 현저하게 작아지게 되어 기울기 소실이 일어난다. 렐루 함수는 은닉층에서 굉장히 많이 사용된다. 별 생각 없이 다층 신경망을 쌓고 은닉층에 어떤 활성화 함수를 써야 할 지 모르겠다 싶으면, 그냥 렐루함수를 쓰면 된다. 
* 기울기 소실이 일어나지 않고 기존 활성화 함수에 비해 속도가 매우 빠르다. 

* `Dense`층에 전달한 매개변수(16)은 은닉 유닉의 개수이다. 하나의 은닉 유닛은 층이 나타내는 표현공간에서 하나의 차원이 된다.
    
    `output = relu(dot(W, input) + b)`

입력 데이터와 w를 점곱하면 편향벡터 b를 더한다. 

`Dense` 층을 쌓을 때 두 가진 중요한 구조상의 결정이 필요합니다:

* 얼마나 많은 층을 사용할 것인가
* 각 층에 얼마나 많은 은닉 유닛을 둘 것인가

4장에서 이런 결정을 하는 데 도움이 되는 일반적인 원리를 배우겠습니다. 당분간은 저를 믿고 선택한 다음 구조를 따라 주세요.

* 16개의 은닉 유닛을 가진 두 개의 은닉층
* 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 세 번째 층

중간에 있는 은닉층은 활성화 함수로 `relu`를 사용하고 마지막 층은 확률(0과 1 사이의 점수로, 어떤 샘플이 타깃 '1'일 가능성이 높다는 것은 그 리뷰가 긍정일 가능성이 높다는 것을 의미합니다)을 출력하기 위해 시그모이드 활성화 함수를 사용합니다. `relu`는 음수를 0으로 만드는 함수입니다. 시그모이드는 임의의 값을 [0, 1] 사이로 압축하므로 출력 값을 확률처럼 해석할 수 있습니다.


In [4]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


2022-01-06 21:00:30.708699: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-06 21:00:30.708749: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-06 21:00:30.708786: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HwangOnyu): /proc/driver/nvidia/version does not exist
2022-01-06 21:00:30.709279: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


손실 함수와 옵티마이저 선택. 이진 분류 문제이고 신경망의 출력이 확률이기 때문에 
`binary_crossentropy` 모델이 적합

`mean_squared_error`라는 모델도 있지만 확률을 출력하는 모델을 사용할 때는 크로스엔트로피가 최선의 선택. 
크로스엔트로피는 정보 이론 분야에서 온 개념으로 확률 분호 간의 차이를 측정한다. 여기에서는 원본 분포와 예측 분포 사이를 측정한다. 

다음은 rmsprop 옵티마이저와 binary_crossentropy 손실 함수로 모델을 설정하는 단계. 훈련하는 동안 정확도를 사용해 모니터링 한다. 

In [8]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy',
                 metrics=['accuracy'])



###  훈련검증

훈련하는 동안 처음 본 데이터에 대한 모델의 정확도를 측정하기 위해서는 원본 훈련 데이터에서 10,000의 샘플을 떼어서 검증 세트를 만들어야 한다. 

In [10]:
x_val = x_train[:10000] #앞의 10000개의 샘플을 떼어 검증 세트를 만든다. 
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

이제 모델을 512개 샘플씩 미니 배치를 만들어 20번의 에포크 동안 훈련시킨다(x_train과 y_train 텐서에 있는 모든 샘플에 대해 20번 반복).
동시에 따로 떼어놓은 10,000개의 샘플에서 손실과 정확도를 측정할 것입니다. 이렇게 하려면 validation_data 매개변수에 검증 데이터를 전달해야 합니다. 

먼저 배치가 무엇인지에 대해서 알아보겠습니다. batch의 사전적 의미에는 "집단, 무리; 한 회분; (일괄 처리를 위해) 함께 묶다" 등이 있습니다. 딥러닝에서 배치는 모델의 가중치를 한번 업데이트시킬 때 사용되는 샘플들의 묶음을 의미합니다. 만약에 총 1000개의 훈련 샘플이 있는데, 배치 사이즈가 20이라면 20개의 샘플 단위마다 모델의 가중치를 한번씩 업데이트시킵니다. 그러니까 총 50번(=1000/20) 가중치가 업데이트되겠죠. 하나의 데이터셋을 총 50개의 배치로 나눠서 훈련을 진행했다고 보면 됩니다.
 

epoch의 사전적 의미는 "(중요한 사건, 변화들이 일어난) 시대"입니다. 딥러닝에서 에포크는 `학습의 횟수`를 의미합니다. 만약 에포크가 10이고 배치 사이즈가 20이면, 가중치를 50번 업데이트하는 것을 총 10번 반복합니다. 각 데이터 샘플이 총 10번씩 사용되는 것입니다. 결과적으로 가중치가 총 500번 업데이트되죠. 

![3-layer network](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FnCJ6J%2FbtqEKREai4u%2FrxMnC8dLCSgkP3FWEt00C0%2Fimg.png)
 

In [11]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',
        metrics=['accuracy'])

history = model.fit(partial_x_train, partial_y_train, 
        epochs=20,batch_size=512,validation_data=(x_val,y_val))

2022-01-06 21:11:27.334161: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 600000000 exceeds 10% of free system memory.


Epoch 1/20
30/30 [==============================] - 3s 46ms/step - loss: 0.5211 - accuracy: 0.7941 - val_loss: 0.3959 - val_accuracy: 0.8595
Epoch 2/20
30/30 [==============================] - 1s 20ms/step - loss: 0.3067 - accuracy: 0.9054 - val_loss: 0.3046 - val_accuracy: 0.8866
Epoch 3/20
30/30 [==============================] - 1s 20ms/step - loss: 0.2251 - accuracy: 0.9261 - val_loss: 0.2847 - val_accuracy: 0.8887
Epoch 4/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1772 - accuracy: 0.9405 - val_loss: 0.2889 - val_accuracy: 0.8828
Epoch 5/20
30/30 [==============================] - 1s 18ms/step - loss: 0.1417 - accuracy: 0.9545 - val_loss: 0.2825 - val_accuracy: 0.8880
Epoch 6/20
30/30 [==============================] - 1s 18ms/step - loss: 0.1183 - accuracy: 0.9633 - val_loss: 0.2949 - val_accuracy: 0.8870
Epoch 7/20
30/30 [==============================] - 1s 19ms/step - loss: 0.0965 - accuracy: 0.9713 - val_loss: 0.3787 - val_accuracy: 0.8694
Epoch 8/20
30

model.fit() 매서드는 History 객체를 반환한다. 이 객체는 훈련하는 동안 발생한 모든 정보를 담고 있는 딕셔너리인 history 속성을 가지고 있다. 

In [13]:
history_dict = history.history
history_dict.keys()


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
import matplotlib.pyplot as plt
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,'bo',label='Training loss') 
plt.plot(epochs,val_loss,'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel("Loss")
plt.legend()

plt.show()

점선은 훈련 손실과 정확도이고 실선은 검증 손실과 정확도입니다. 신경망의 무작위한 초기화 때문에 사람마다 결과가 조금 다를 수 있습니다.

여기에서 볼 수 있듯이 훈련 손실이 에포크마다 감소하고 훈련 정확도는 에포크마다 증가합니다. 경사 하강법 최적화를 사용했을 때 반복마다 최소화되는 것이 손실이므로 기대했던 대로입니다. 검증 손실과 정확도는 이와 같지 않습니다. 4번째 에포크에서 그래프가 역전되는 것 같습니다. 이것이 훈련 세트에서 잘 작동하는 모델이 처음 보는 데이터에 잘 작동하지 않을 수 있다고 앞서 언급한 경고의 한 사례입니다. 정확한 용어로 말하면 과대적합되었다고 합니다. 2번째 에포크 이후부터 훈련 데이터에 과도하게 최적화되어 훈련 데이터에 특화된 표현을 학습하므로 훈련 세트 이외의 데이터에는 일반화되지 못합니다.

이런 경우에 과대적합을 방지하기 위해서 3번째 에포크 이후에 훈련을 중지할 수 있습니다. 일반적으로 4장에서 보게 될 과대적합을 완화하는 다양한 종류의 기술을 사용할 수 있습니다.

처음부터 다시 새로운 신경망을 4번의 에포크 동안만 훈련하고 테스트 데이터에서 평가해 보겠습니다:

In [14]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

2022-01-06 22:15:38.902775: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1000000000 exceeds 10% of free system memory.


Epoch 1/4
49/49 [==============================] - 3s 37ms/step - loss: 0.4783 - accuracy: 0.8150
Epoch 2/4
49/49 [==============================] - 2s 34ms/step - loss: 0.2726 - accuracy: 0.9108
Epoch 3/4
49/49 [==============================] - 2s 35ms/step - loss: 0.2063 - accuracy: 0.9288
Epoch 4/4
49/49 [==============================] - 2s 31ms/step - loss: 0.1667 - accuracy: 0.9416


2022-01-06 22:15:57.039688: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1000000000 exceeds 10% of free system memory.


782/782 [==============================] - 4s 4ms/step - loss: 0.2916 - accuracy: 0.8850


In [15]:
 results

[0.2915942072868347, 0.8849999904632568]

87퍼센트의 정확도 달성. 

## 훈련된 모델로 새로운 데이터에 대해 예측하기

모델을 훈련시킨 후에 이를 실전 환경에서 사용하고 싶을 것입니다. `predict` 메서드를 사용해서 어떤 리뷰가 긍정일 확률을 예측할 수 있습니다:

In [16]:
model.predict(x_test)

2022-01-06 22:18:30.946616: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1000000000 exceeds 10% of free system memory.


array([[0.15424672],
       [0.9979833 ],
       [0.6156576 ],
       ...,
       [0.11574012],
       [0.08164006],
       [0.59053445]], dtype=float32)

여기에서처럼 이 모델은 어떤 샘플에 대해 확신을 가지고 있지만(0.99 또는 그 이상, 0.01 또는 그 이하) 어떤 샘플에 대해서는 확신이 부족합니다(0.6, 0.4). 

## 추가 실험

* 여기에서는 두 개의 은닉층을 사용했습니다. 한 개 또는 세 개의 은닉층을 사용하고 검증과 테스트 정확도에 어떤 영향을 미치는지 확인해 보세요.
* 층의 은닉 유닛을 추가하거나 줄여 보세요: 32개 유닛, 64개 유닛 등
* `binary_crossentropy` 대신에 `mse` 손실 함수를 사용해 보세요.
* `relu` 대신에 `tanh` 활성화 함수(초창기 신경망에서 인기 있었던 함수입니다)를 사용해 보세요.

다음 실험을 진행하면 여기에서 선택한 구조가 향상의 여지는 있지만 어느 정도 납득할 만한 수준이라는 것을 알게 것입니다!

## 정리

다음은 이 예제에서 배운 것들입니다:

* 원본 데이터를 신경망에 텐서로 주입하기 위해서는 꽤 많은 전처리가 필요합니다. 단어 시퀀스는 이진 벡터로 인코딩될 수 있고 다른 인코딩 방식도 있습니다.
* `relu` 활성화 함수와 함께 `Dense` 층을 쌓은 네트워크는 (감성 분류를 포함하여) 여러 종류의 문제에 적용할 수 있어서 앞으로 자주 사용하게 될 것입니다.
* (출력 클래스가 두 개인) 이진 분류 문제에서 네트워크는 하나의 유닛과 `sigmoid` 활성화 함수를 가진 `Dense` 층으로 끝나야 합니다. 이 신경망의 출력은 확률을 나타내는 0과 1 사이의 스칼라 값입니다.
* 이진 분류 문제에서 이런 스칼라 시그모이드 출력에 대해 사용할 손실 함수는 `binary_crossentropy`입니다.
* `rmsprop` 옵티마이저는 문제에 상관없이 일반적으로 충분히 좋은 선택입니다. 걱정할 거리가 하나 줄은 셈입니다.
* 훈련 데이터에 대해 성능이 향상됨에 따라 신경망은 과대적합되기 시작하고 이전에 본적 없는 데이터에서는 결과가 점점 나빠지게 됩니다. 항상 훈련 세트 이외의 데이터에서 성능을 모니터링해야 합니다.